In [4]:
from typing import Tuple
import pandas as pd
from tqdm.notebook import tqdm
import sys
sys.path.append('..')
from evaluation.generated_dataset import GeneratedDataset, load_all_from_config
from evaluation.novelty import NoveltyFilter, filter_by_unique_structure

In [6]:
raw_datasets = {
    "WyFormer": ("WyckoffTransformer", "CrySPR", "CHGNet_fix"),
#    "DiffCSP": ("DiffCSP",)
}

In [7]:
all_datasets = load_all_from_config(
    datasets=list(raw_datasets.values()) + \
        [("split", "train"), ("split", "val"), ("split", "test")],
    dataset_name="perov_5")

In [8]:
novelty_reference = pd.concat([
    all_datasets[('split', 'train')].data,
    all_datasets[('split', 'val')].data], axis=0, verify_integrity=True)
novelty_filter = NoveltyFilter(novelty_reference)

In [10]:
import evaluation.statistical_evaluator
test_evaluator = evaluation.statistical_evaluator.StatisticalEvaluator(all_datasets[('split', 'test')].data)

In [13]:
import evaluation.novelty
train_w_template_set = frozenset(novelty_reference.apply(evaluation.novelty.record_to_anonymous_fingerprint, axis=1))
train_strict_AFLOW_set = frozenset(novelty_reference.apply(evaluation.novelty.record_to_strict_AFLOW_fingerprint, axis=1))
train_relaxed_AFLOW_set = frozenset(novelty_reference.apply(evaluation.novelty.record_to_relaxed_AFLOW_fingerprint, axis=1))

In [14]:
perov_5 = pd.concat([
    all_datasets[('split', 'train')].data,
    all_datasets[('split', 'val')].data,
    all_datasets[('split', 'test')].data], axis=0, verify_integrity=True)

In [17]:
perov_5.smact_validity.mean()

0.9867392223161454

In [21]:
(perov_5.spacegroup_number == 1).any()

False

In [24]:
perov_5.spacegroup_number.value_counts()

spacegroup_number
123    5633
25     4310
99     3632
221    3122
47     2231
Name: count, dtype: int64

In [27]:
table = pd.DataFrame(
    index=raw_datasets.keys(), columns=[
        "Novelty (%)", "Structural", "Compositional", 
        "Recall", "Precision",
        r"$\rho$", "$E$", "# Elements",
        "Novel Template (%)", 
        "Novel AFLOW on P1",
        "Novel AFLOW on !P1",
        "Novel AFLOW (%)",
        "Novel AFLOW relaxed (%)",
        "P1 (%)",
        "Space Group"])
table.index.name = "Method"
E_hull_threshold = 0.08
for name, transformations in tqdm(raw_datasets.items()):
    dataset = all_datasets[transformations]
    unique = filter_by_unique_structure(dataset.data)
    print(f"Unique: {len(unique)} / {len(dataset.data)} = {len(unique) / len(dataset.data)}")
    novel_template = ~unique.apply(evaluation.novelty.record_to_anonymous_fingerprint, axis=1).isin(train_w_template_set)
    table.loc[name, "Novel Template (%)"] = 100 * novel_template.mean()
    novel_aflow_strict = ~unique.apply(evaluation.novelty.record_to_strict_AFLOW_fingerprint, axis=1).isin(train_strict_AFLOW_set)
    table.loc[name, "Novel AFLOW (%)"] = 100 * novel_aflow_strict.mean()
    novel_aflow_relaxed = ~unique.apply(evaluation.novelty.record_to_relaxed_AFLOW_fingerprint, axis=1).isin(train_relaxed_AFLOW_set)
    table.loc[name, "Novel AFLOW relaxed (%)"] = 100 * novel_aflow_relaxed.mean()
    unique_is_P1 = unique.group == 1
    table.loc[name, "Novel AFLOW on P1"] = 100 * novel_aflow_strict[unique_is_P1].mean()
    table.loc[name, "Novel AFLOW on !P1"] = 100 * novel_aflow_strict[~unique_is_P1].mean()
    if transformations == ("split", "train"):
        novel = unique
    else:
        novel = novelty_filter.get_novel(unique)
    table.loc[name, "Novelty (%)"] = 100 * len(novel) / len(unique)
    if "structural_validity" in novel.columns:
        table.loc[name, "Structural"] = 100 * novel.structural_validity.mean()
        table.loc[name, "Compositional"] = 100 * novel.smact_validity.mean()
    if "cdvae_crystal" in novel.columns:
        cov_metrics = test_evaluator.get_coverage(novel.cdvae_crystal)    
        table.loc[name, "Recall"] = 100 * cov_metrics["cov_recall"]
        table.loc[name, "Precision"] = 100 * cov_metrics["cov_precision"]
        novel = novel[novel.structural_validity]
        table.loc[name, r"$\rho$"] = test_evaluator.get_density_emd(novel)
        table.loc[name, "$E$"] = test_evaluator.get_cdvae_e_emd(novel)
        table.loc[name, "# Elements"] = test_evaluator.get_num_elements_emd(novel)
    table.loc[name, "P1 (%)"] = 100 * (novel.group == 1).mean()
    table.loc[name, "Space Group"] = test_evaluator.get_sg_chi2(novel)
table

  0%|          | 0/1 [00:00<?, ?it/s]

Unique: 992 / 1000 = 0.992


,Novelty (%),Structural,Compositional,Recall,Precision,$\rho$,$E$,# Elements,Novel Template (%),Novel AFLOW on P1,Novel AFLOW on !P1,Novel AFLOW (%),Novel AFLOW relaxed (%),P1 (%),Space Group
Method,,,,,,,,,,,,,,,
WyFormer,62.5,100.0,98.548387,91.290323,99.603699,0.132252,0.084367,0.05786,0.201613,NaN,0.302419,0.302419,0.302419,0.0,0.109391


In [33]:
cdvae_table = pd.DataFrame(index=pd.Index(raw_datasets.keys(), tupleize_cols=False),
    columns=[
        "Structural", "Compositional",
        "Recall", "Precision",
        r"$\rho$", "$E$", "# Elements"])
sample_size = 992
for name, transformations in tqdm(raw_datasets.items()):
    dataset = all_datasets[transformations]
    if "structure" in dataset.data.columns:
        cdvae_table.loc[name, "Compositional"] = 100*dataset.data.smact_validity.mean()
        cdvae_table.loc[name, "Structural"] = 100*dataset.data.structural_validity.mean()
        valid = dataset.data[dataset.data.naive_validity]
        cov_metrics = test_evaluator.get_coverage(valid.cdvae_crystal)
        cdvae_table.loc[name, "Recall"] = 100*cov_metrics["cov_recall"]
        cdvae_table.loc[name, "Precision"] = 100*cov_metrics["cov_precision"]
        cdvae_table.loc[name, r"$\rho$"] = test_evaluator.get_density_emd(valid)
        cdvae_table.loc[name, "$E$"] = test_evaluator.get_cdvae_e_emd(valid)
        cdvae_table.loc[name, "# Elements"] = test_evaluator.get_num_elements_emd(valid)
cdvae_table.to_csv("tables/cdvae_metrics_no_relax_table.csv")
#prettify(cdvae_table).to_latex("tables/cdvae_metrics_no_relax_table.tex", siunitx=True, convert_css=True)
#prettify(cdvae_table)

  0%|          | 0/1 [00:00<?, ?it/s]

In [34]:
cdvae_table

,Structural,Compositional,Recall,Precision,$\rho$,$E$,# Elements
WyFormer,100.0,98.2,94.501018,99.709379,0.132511,0.049105,0.041792
